In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
seed = 2
from xgboost import XGBClassifier
import pickle

In [2]:
train = pd.read_csv('./data/new_train.csv')

In [3]:
sample_size = int(1e4)
sample = train[train.target == 1].sample(n=sample_size/2, random_state=seed)
sample = sample.append( train[train.target == 0].sample(n=sample_size/2, random_state=seed))
sample = sample.sample(frac=1, random_state=seed).reset_index(drop=True)
#sample.to_csv('./data/train_sample.csv', index=False)

In [4]:
target = train.target
train.drop(['target','id'], inplace=True, axis=1)
train.drop([ col for col in train.columns if col.startswith('ps_cont') ],axis=1, inplace=True)

In [5]:
with open('./data/OneHotEncoder.clf', 'rb') as f:
    encoders = pickle.load(f)

In [6]:
enc_train = None

for feature,encoder in zip(train.columns,encoders) :
    encoded = encoder.transform(train[feature].values.reshape(-1,1))
    if enc_train is None :
        enc_train = encoded
    else :
        enc_train = np.concatenate((enc_train, encoded), axis=1)

In [7]:
del train

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(enc_train, target, test_size=.1, random_state=seed)

In [ ]:
model = XGBClassifier(
	objective = 'binary:logistic',
	n_jobs = 4,
	random_state = seed,
	eval_metric = "error",
	#-------
	max_depth = 1,
	min_child_weight = 7,
	#----------
	gamma = 0,
	#----------
	subsample = 0.4,
	colsample_bytree = 0.4,
	#------------
	scale_pos_weight = 3,
	#-----------
	reg_alpha = 0,
	reg_lambda = 1,
	#------------
	learning_rate = 0.1

	#n_estimators = 1000,
	)

In [ ]:
model.fit(enc_train, target)

In [ ]:
with open('./data/model.pkl', 'wb') as f:
    pickle.dump(file=f, obj=model)

In [8]:
with open('./data/model.pkl', 'rb') as f:
    model = pickle.load(f)

In [9]:
pred = model.predict(enc_train)

In [10]:
from sklearn.metrics import classification_report
print classification_report(y_true=target, y_pred=pred)

             precision    recall  f1-score   support

          0       0.96      1.00      0.98    573518
          1       0.00      0.00      0.00     21694

avg / total       0.93      0.96      0.95    595212



/usr/lib64/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
train.ps_cat_ind_02.value_counts()

In [ ]:
del x_train, y_train